In [1]:
import tensorflow_datasets as tfds

In [2]:
import tensorflow as tf

In [3]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompletePGBG6M/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompletePGBG6M/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompletePGBG6M/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [4]:
train_dataset, test_dataset = dataset['train'], dataset['test']

In [5]:
len(train_dataset)

25000

In [6]:
import numpy as np


Các thông số của mô hình MLP Mixer. Chi tiết xem thêm [tại đây](https://www.youtube.com/watch?v=T1hGxK5IsIA&t=1852s)

In [7]:
S = 300
C = 512
DS = 256
DC = 2048
num_of_mlp_blocks = 8
num_classes = 2

In [8]:
max_length = S

Các câu có chiều dài không đều nhau, để tiết kiệm thời gian chúng tôi đã lấy các câu ra khỏi tf dataset để padding và thử nghiệm nhanh. Sẽ tốt hơn nếu sử dụng hàm `map` để thao tác với `tf dataset` 

In [9]:
def process_data(ds, max_length):
  num_items = len(ds)
  X = np.zeros((num_items, max_length), dtype=np.int64)
  Y = np.zeros((num_items,), dtype=np.int64)
  for i, item in enumerate(ds):
    x, y = item
    x = x.numpy()
    y = y.numpy()
    
    x = np.reshape(x, (1, x.shape[0]))
    x_padded = np.zeros((1, max_length), dtype=np.int64)

    if x.shape[1] < max_length:
      x_padded[:, :x.shape[1]] = x
    else:
      x_padded[:, :] = x[:, :max_length]

    X[i, :] = x_padded
    Y[i] = y 
  return X, Y

X_train, Y_train = process_data(train_dataset, max_length)
X_val, Y_val = process_data(test_dataset, max_length)

Kiểm tra chiều

In [10]:
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape

((25000, 300), (25000,), (25000, 300), (25000,))

In [11]:
vocab_size=8185

In [12]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [16]:
from tensorflow.keras.layers import Embedding, Input, LayerNormalization, Dense, GlobalAveragePooling1D, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Normalization, Resizing, RandomFlip, RandomRotation, RandomZoom

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Normalization, Resizing, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding

MLP Block này được giữ nguyên theo code [tại đây](https://github.com/bangoc123/mlp-mixer)

In [17]:
class MLPBlock(tf.keras.layers.Layer):
  def __init__(self, S, C, DS, DC):
    super(MLPBlock, self).__init__()
    self.layerNorm1 = LayerNormalization()
    self.layerNorm2 = LayerNormalization()
    w_init = tf.random_normal_initializer()
    self.DS = DS
    self.DC = DC
    self.W1 = tf.Variable(
            initial_value=w_init(shape=(S, DS), dtype="float32"),
            trainable=True,
    )
    self.W2 = tf.Variable(
            initial_value=w_init(shape=(DS, S), dtype="float32"),
            trainable=True,
    )
    self.W3 = tf.Variable(
            initial_value=w_init(shape=(C, DC), dtype="float32"),
            trainable=True,
    )
    self.W4 = tf.Variable(
            initial_value=w_init(shape=(DC, C), dtype="float32"),
            trainable=True,
    )

  def call(self, X):
    # patches (..., S, C)
    batch_size, S, C = X.shape
    
    # Token-mixing
    # (..., C, S)
    X_T = tf.transpose(self.layerNorm1(X), perm=(0, 2, 1))

    # assert X_T.shape == (batch_size, C, S), 'X_T.shape: {}'.format(X_T.shape)

    W1X = tf.matmul(X_T, self.W1) # (..., C, S) . (S, DS) = (..., C, DS)

    # (..., C, DS) . (DS, S) == (..., C, S)
    # (..., C, S). T == (..., S, C)
    # (..., S, C) + (..., S, C) = (..., S, C)
    U = tf.transpose(tf.matmul(tf.nn.gelu(W1X), self.W2), perm=(0, 2, 1)) + X

    # Channel-minxing
    
    W3U = tf.matmul(self.layerNorm2(U), self.W3) # (...,S, C) . (C, DC) = (..., S, DC)

    Y = tf.matmul(tf.nn.gelu(W3U), self.W4) + U  # (..., S, DC) . (..., DC, C) + (..., S, C) = (..., S, C)

    return Y

In [18]:
vocab_size

8185

Thay đối kiến trúc thay vì chia các Patch ảnh thì chỉ sử dụng Embedding để số hóa các từ theo các quy trình thông thường.

In [ ]:

class SentimentMixer(tf.keras.models.Model):
  def __init__(self, S, C, DS, DC, num_of_mlp_blocks, vocab_size, num_classes):
    super(SentimentMixer, self).__init__()
    self.projection = Dense(C)
    self.mlpBlocks = [MLPBlock(S, C, DS, DC) for _ in range(num_of_mlp_blocks)]
    
    self.S = S
    self.C = C
    self.DS = DS
    self.DC = DC
    self.num_classes = num_classes
    self.word_embedding = Embedding(vocab_size, output_dim=self.C)
    self.classificationLayer = Sequential([
          GlobalAveragePooling1D(),
          Dropout(0.2),
          Dense(num_classes, activation='softmax')
    ])

  def call(self, X):

    embedded_x = self.word_embedding(X)
    mlp_input = self.projection(embedded_x)

    for block in self.mlpBlocks:
      mlp_input = block(mlp_input)
      
    out = self.classificationLayer(mlp_input)
    return out


sentimentMixer = SentimentMixer(S, C, DS, DC, num_of_mlp_blocks, vocab_size, num_classes)
learning_rate = 0.01
epochs = 10


# Set up loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

# Optimizer Definition
adam = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compile optimizer and loss function into model
sentimentMixer.compile(optimizer=adam, loss=loss_object, metrics=['acc'])


sentimentMixer.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_val, Y_val))

Epoch 1/10
782/782 [==============================] - 278s 343ms/step - loss: 7.3602 - acc: 0.6018 - val_loss: 0.6635 - val_acc: 0.7030
Epoch 2/10
782/782 [==============================] - 267s 342ms/step - loss: 4.4536 - acc: 0.7578 - val_loss: 0.4684 - val_acc: 0.7808
Epoch 3/10
782/782 [==============================] - 267s 341ms/step - loss: 0.3556 - acc: 0.8479 - val_loss: 0.4567 - val_acc: 0.7947
Epoch 4/10
158/782 [=====>........................] - ETA: 2:42 - loss: 0.2589 - acc: 0.9021